In [1]:
import pandas as pd
import numpy as np
import optuna

In [2]:
df = pd.read_csv('/kaggle/input/ml-dataset/Final_Anomaly_Removed_Data.csv')

In [3]:
# Normalization function in range [0, 1]
def normalize_column(column):
    min_val = column.min()
    max_val = column.max()
    return (column - min_val) / (max_val - min_val)

# Inverse normalization function
def inverse_normalize_column(norm_column, min_val, max_val):
    return norm_column * (max_val - min_val) + min_val

# Normalize the data
normalized_data = df.apply(normalize_column)

# Reverse the normalization (Example with one column)
# original_data = inverse_normalize_column(normalized_data['column_name'], df['column_name'].min(), df['column_name'].max())

In [4]:
from sklearn.model_selection import train_test_split
train, test = train_test_split(normalized_data, test_size=0.2, random_state=42)

In [5]:
normalized_data.drop(columns = ['%SI','%FE','%TI','%V','%AL'],inplace = True)

In [6]:
normalized_data.describe()

,Unnamed: 0,EMUL_OIL_L_TEMP_PV_VAL0,STAND_OIL_L_TEMP_PV_REAL_VAL0,GEAR_OIL_L_TEMP_PV_REAL_VAL0,EMUL_OIL_L_PR_VAL0,QUENCH_CW_FLOW_EXIT_VAL0,CAST_WHEEL_RPM_VAL0,BAR_TEMP_VAL0,QUENCH_CW_FLOW_ENTRY_VAL0,GEAR_OIL_L_PR_VAL0,STANDS_OIL_L_PR_VAL0,TUNDISH_TEMP_VAL0,RM_MOTOR_COOL_WATER__VAL0,ROLL_MILL_AMPS_VAL0,RM_COOL_WATER_FLOW_VAL0,EMULSION_LEVEL_ANALO_VAL0,UTS,Elongation,Conductivity,Furnace_Temperature
count,4499.000000,4499.000000,4499.000000,4499.000000,4499.000000,4499.000000,4499.000000,4499.000000,4499.000000,4499.000000,4499.000000,4499.000000,4499.000000,4499.000000,4499.000000,4499.000000,4499.000000,4499.000000,4499.000000,4499.000000
mean,0.500435,0.943989,0.725397,0.576122,0.387188,0.464280,0.756651,0.906696,0.489952,0.456159,0.654117,0.662170,0.362325,0.305570,0.536631,0.662680,0.611074,0.426861,0.682462,0.508853
std,0.288728,0.075345,0.162935,0.169425,0.296794,0.170303,0.119367,0.093658,0.203231,0.174689,0.198654,0.046984,0.153866,0.117729,0.208663,0.065614,0.062939,0.155514,0.174386,0.068341
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.251207,0.944619,0.665961,0.479983,0.127538,0.358153,0.705392,0.872431,0.316955,0.348854,0.463983,0.654596,0.253278,0.233598,0.405166,0.631505,0.567568,0.345455,0.608696,0.497410
50%,0.498903,0.959012,0.777367,0.585728,0.251088,0.473989,0.774660,0.921104,0.412572,0.450474,0.674308,0.660863,0.361740,0.299566,0.517909,0.649003,0.601351,0.436364,0.695652,0.511821
75%,0.752305,0.965134,0.839913,0.687283,0.655758,0.572933,0.842613,0.968094,0.663360,0.559149,0.802691,0.669591,0.442717,0.373555,0.721221,0.684182,0.641892,0.540909,0.826087,0.528689
max,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [7]:
features = train[['EMUL_OIL_L_TEMP_PV_VAL0','STAND_OIL_L_TEMP_PV_REAL_VAL0','GEAR_OIL_L_TEMP_PV_REAL_VAL0','EMUL_OIL_L_PR_VAL0','QUENCH_CW_FLOW_EXIT_VAL0','CAST_WHEEL_RPM_VAL0','BAR_TEMP_VAL0','QUENCH_CW_FLOW_ENTRY_VAL0','GEAR_OIL_L_PR_VAL0','STANDS_OIL_L_PR_VAL0','TUNDISH_TEMP_VAL0', 'RM_MOTOR_COOL_WATER__VAL0', 'ROLL_MILL_AMPS_VAL0', 'RM_COOL_WATER_FLOW_VAL0','EMULSION_LEVEL_ANALO_VAL0','Furnace_Temperature']].to_numpy()
labels = train[['   UTS','Elongation','Conductivity']].to_numpy()

print("Features shape:", features.shape)
print("Labels shape:", labels.shape)

Features shape: (3599, 16)
Labels shape: (3599, 3)


In [8]:
features_test = test[['EMUL_OIL_L_TEMP_PV_VAL0','STAND_OIL_L_TEMP_PV_REAL_VAL0','GEAR_OIL_L_TEMP_PV_REAL_VAL0','EMUL_OIL_L_PR_VAL0','QUENCH_CW_FLOW_EXIT_VAL0','CAST_WHEEL_RPM_VAL0','BAR_TEMP_VAL0','QUENCH_CW_FLOW_ENTRY_VAL0','GEAR_OIL_L_PR_VAL0','STANDS_OIL_L_PR_VAL0','TUNDISH_TEMP_VAL0', 'RM_MOTOR_COOL_WATER__VAL0', 'ROLL_MILL_AMPS_VAL0', 'RM_COOL_WATER_FLOW_VAL0','EMULSION_LEVEL_ANALO_VAL0','Furnace_Temperature']].to_numpy()
labels_test = test[['   UTS','Elongation','Conductivity']].to_numpy()

print("Features shape:", features_test.shape)
print("Labels shape:", labels_test.shape)

Features shape: (900, 16)
Labels shape: (900, 3)


In [9]:
normalized_data.describe()

,Unnamed: 0,EMUL_OIL_L_TEMP_PV_VAL0,STAND_OIL_L_TEMP_PV_REAL_VAL0,GEAR_OIL_L_TEMP_PV_REAL_VAL0,EMUL_OIL_L_PR_VAL0,QUENCH_CW_FLOW_EXIT_VAL0,CAST_WHEEL_RPM_VAL0,BAR_TEMP_VAL0,QUENCH_CW_FLOW_ENTRY_VAL0,GEAR_OIL_L_PR_VAL0,STANDS_OIL_L_PR_VAL0,TUNDISH_TEMP_VAL0,RM_MOTOR_COOL_WATER__VAL0,ROLL_MILL_AMPS_VAL0,RM_COOL_WATER_FLOW_VAL0,EMULSION_LEVEL_ANALO_VAL0,UTS,Elongation,Conductivity,Furnace_Temperature
count,4499.000000,4499.000000,4499.000000,4499.000000,4499.000000,4499.000000,4499.000000,4499.000000,4499.000000,4499.000000,4499.000000,4499.000000,4499.000000,4499.000000,4499.000000,4499.000000,4499.000000,4499.000000,4499.000000,4499.000000
mean,0.500435,0.943989,0.725397,0.576122,0.387188,0.464280,0.756651,0.906696,0.489952,0.456159,0.654117,0.662170,0.362325,0.305570,0.536631,0.662680,0.611074,0.426861,0.682462,0.508853
std,0.288728,0.075345,0.162935,0.169425,0.296794,0.170303,0.119367,0.093658,0.203231,0.174689,0.198654,0.046984,0.153866,0.117729,0.208663,0.065614,0.062939,0.155514,0.174386,0.068341
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.251207,0.944619,0.665961,0.479983,0.127538,0.358153,0.705392,0.872431,0.316955,0.348854,0.463983,0.654596,0.253278,0.233598,0.405166,0.631505,0.567568,0.345455,0.608696,0.497410
50%,0.498903,0.959012,0.777367,0.585728,0.251088,0.473989,0.774660,0.921104,0.412572,0.450474,0.674308,0.660863,0.361740,0.299566,0.517909,0.649003,0.601351,0.436364,0.695652,0.511821
75%,0.752305,0.965134,0.839913,0.687283,0.655758,0.572933,0.842613,0.968094,0.663360,0.559149,0.802691,0.669591,0.442717,0.373555,0.721221,0.684182,0.641892,0.540909,0.826087,0.528689
max,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [10]:
normalized_data.drop(columns = ['Unnamed: 0'],inplace = True)

In [11]:
normalized_data = normalized_data.astype('float32')

In [12]:
normalized_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4499 entries, 0 to 4498
Data columns (total 19 columns):
 #   Column                         Non-Null Count  Dtype  
---  ------                         --------------  -----  
 0   EMUL_OIL_L_TEMP_PV_VAL0        4499 non-null   float32
 1   STAND_OIL_L_TEMP_PV_REAL_VAL0  4499 non-null   float32
 2   GEAR_OIL_L_TEMP_PV_REAL_VAL0   4499 non-null   float32
 3   EMUL_OIL_L_PR_VAL0             4499 non-null   float32
 4   QUENCH_CW_FLOW_EXIT_VAL0       4499 non-null   float32
 5   CAST_WHEEL_RPM_VAL0            4499 non-null   float32
 6   BAR_TEMP_VAL0                  4499 non-null   float32
 7   QUENCH_CW_FLOW_ENTRY_VAL0      4499 non-null   float32
 8   GEAR_OIL_L_PR_VAL0             4499 non-null   float32
 9   STANDS_OIL_L_PR_VAL0           4499 non-null   float32
 10  TUNDISH_TEMP_VAL0              4499 non-null   float32
 11  RM_MOTOR_COOL_WATER__VAL0      4499 non-null   float32
 12  ROLL_MILL_AMPS_VAL0            4499 non-null   f

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
from tqdm import tqdm
import random, os, numpy as np

# Configuration class
class Config:
    DEVICE = "cuda:0"
    TRAIN_DIR = "data/train"
    VAL_DIR = "data/val"
    BATCH_SIZE = 16
    NUM_WORKERS = 4
    LEARNING_RATE = 1e-6
    LAMBDA_CYCLE = 12.625839019572867
    NUM_EPOCHS = 100
    LOAD_MODEL = False
    SAVE_MODEL = True
    CHECKPOINT_GEN_INPUT = "gen_input.pth.tar"
    CHECKPOINT_GEN_OUTPUT = "gen_output.pth.tar"
    CHECKPOINT_DISC_INPUT = "disc_input.pth.tar"
    CHECKPOINT_DISC_OUTPUT = "disc_output.pth.tar"
    DROPOUT_RATE = 0.0
    MEAN = 0.0
    STD = 1.0
    PRINT_FREQ = 100
    SAVE_FREQ = 1

class Generator(nn.Module):
    def __init__(self, input_dim, output_dim):
        super(Generator, self).__init__()
        self.model = nn.Sequential(
            nn.Linear(input_dim, 128),
            nn.InstanceNorm1d(128),  # Batch normalization for stable training
            nn.LeakyReLU(0.2, inplace=True),
            nn.Linear(128, 256),
            nn.InstanceNorm1d(256),  # Batch normalization for stable training
            nn.LeakyReLU(0.2, inplace=True),
            nn.Linear(256, 128),
            nn.InstanceNorm1d(128),  # Batch normalization for stable training
            nn.LeakyReLU(0.2, inplace=True),
            nn.Linear(128, output_dim)
        )
        # Skip connection for better gradient flow

    def forward(self, x):
        return self.model(x)

# Discriminator definition
class Discriminator(nn.Module):
    def __init__(self, input_dim):
        super(Discriminator, self).__init__()
        self.model = nn.Sequential(
            nn.Linear(input_dim, 128),
            nn.InstanceNorm1d(128),  # Batch normalization for stable training
            nn.LeakyReLU(0.2, inplace=True),
            nn.Dropout(0.3),  # Dropout for regularization
            nn.Linear(128, 64),
            nn.InstanceNorm1d(64),  # Batch normalization for stable training
            nn.LeakyReLU(0.2, inplace=True),
            nn.Dropout(0.3),
            nn.Linear(64, 1),  # Single node for real/fake classification
            nn.Sigmoid()
        )

    def forward(self, x):
        return self.model(x)

# Feature dataset class definition
class FeatureDataset(Dataset):
    def __init__(self, inputs, outputs):
        self.inputs = torch.tensor(inputs, dtype=torch.float32)
        self.outputs = torch.tensor(outputs, dtype=torch.float32)

    def __len__(self):
        return len(self.inputs)

    def __getitem__(self, idx):
        return self.inputs[idx], self.outputs[idx]

# Training function definition
def train_fn(disc_input, disc_output, gen_input, gen_output,
             loader, opt_disc, opt_gen, l1_loss_fn,
             mse_loss_fn, d_scaler, g_scaler):

    loop = tqdm(loader)

    for idx, (inputs, outputs) in enumerate(loop):
        inputs = inputs.to(Config.DEVICE)
        outputs = outputs.to(Config.DEVICE)

        # Train Discriminators
        with torch.amp.autocast(device_type = Config.DEVICE, dtype = torch.float32):
            fake_inputs = gen_input(outputs)        #Assumption [32,26]

            D_input_real = disc_input(inputs)       #Correct
            D_input_fake = disc_input(fake_inputs.detach()) # Correct

            D_input_real_loss = mse_loss_fn(D_input_real, torch.ones_like(D_input_real))  #Correct
            D_input_fake_loss = mse_loss_fn(D_input_fake, torch.zeros_like(D_input_fake)) #Correct

            D_input_loss = D_input_real_loss + D_input_fake_loss

            fake_outputs = gen_output(inputs)       #Assumption [32,3]

            D_output_real = disc_output(outputs)    #Correct
            D_output_fake = disc_output(fake_outputs.detach())    #Correct

            D_output_real_loss = mse_loss_fn(D_output_real, torch.ones_like(D_output_real))   #Correct
            D_output_fake_loss = mse_loss_fn(D_output_fake, torch.zeros_like(D_output_fake))    #Correct

            D_output_loss = D_output_real_loss + D_output_fake_loss

            D_loss = (D_input_loss + D_output_loss) / 2

        opt_disc.zero_grad()
        d_scaler.scale(D_loss).backward()
        d_scaler.step(opt_disc)
        d_scaler.update()

        # Train Generators
        with torch.amp.autocast(device_type = Config.DEVICE, dtype = torch.float32):
            D_input_fake = disc_input(fake_inputs)
            D_output_fake = disc_output(fake_outputs)

            loss_G_input = mse_loss_fn(D_input_fake, torch.ones_like(D_input_fake))
            loss_G_output = mse_loss_fn(D_output_fake, torch.ones_like(D_output_fake))

            # Cycle loss
            cycle_inputs = gen_input(fake_outputs)
            cycle_outputs = gen_output(fake_inputs)

            cycle_input_loss = l1_loss_fn(inputs, cycle_inputs)
            cycle_output_loss = l1_loss_fn(outputs, cycle_outputs)

            # Combine losses
            G_loss = (
                loss_G_input +
                loss_G_output +
                cycle_input_loss * Config.LAMBDA_CYCLE +
                cycle_output_loss * Config.LAMBDA_CYCLE
            )

        opt_gen.zero_grad()
        g_scaler.scale(G_loss).backward()
        g_scaler.step(opt_gen)
        g_scaler.update()

# Checkpoint saving/loading functions
def save_checkpoint(model, optimizer, filename="my_checkpoint.pth.tar"):
    print("=> Saving checkpoint")
    checkpoint = {
      "state_dict": model.state_dict(),
      "optimizer": optimizer.state_dict(),
    }
    torch.save(checkpoint, filename)

def load_checkpoint(checkpoint_file, model, optimizer):
    print("=> Loading checkpoint")
    checkpoint = torch.load(checkpoint_file)

    model.load_state_dict(checkpoint["state_dict"])

    optimizer.load_state_dict(checkpoint["optimizer"])

def seed_everything(seed=42):
    os.environ["PYTHONHASHSEED"] = str(seed)
    random.seed(seed)
    np.random.seed(seed)

    torch.manual_seed(seed)


def mean_deviation(y_true, y_pred):
    absolute_deviation = torch.abs(y_true - y_pred)  # Calculate the absolute deviations
    mean_dev = torch.mean(absolute_deviation)       # Average the deviations
    return mean_dev.item()  # Convert tensor to Python float

model_save_path = { 'gen_input': '/kaggle/working/gen_input.pth.tar', 'gen_output': '/kaggle/working/gen_output.pth.tar', 'disc_input': '/kaggle/working/disc_input.pth.tar', 'disc_output': '/kaggle/working/disc_output.pth.tar' }
md_score_best = 100
def objective(trial):
    global md_score_best
    input_dim, output_dim = 16, 3

    # Suggest hyperparameters
    learning_rate = trial.suggest_float('learning_rate', 1e-5, 1e-3, log=True)
    batch_size = trial.suggest_int('batch_size', 8, 64)
    num_epochs = trial.suggest_int('num_epochs', 10, 200)
    lambda_cycle = trial.suggest_float('lambda_cycle', 1, 20)

    # Update Config with the suggested hyperparameters
    Config.LEARNING_RATE = learning_rate
    Config.BATCH_SIZE = batch_size
    Config.NUM_EPOCHS = num_epochs
    Config.LAMBDA_CYCLE = lambda_cycle

    # Initialize models
    disc_input = Discriminator(input_dim=input_dim).to(Config.DEVICE)
    disc_output = Discriminator(input_dim=output_dim).to(Config.DEVICE)
    gen_input = Generator(input_dim=output_dim, output_dim=input_dim).to(Config.DEVICE)
    gen_output = Generator(input_dim=input_dim, output_dim=output_dim).to(Config.DEVICE)

    # Optimizers
    opt_disc = optim.Adam(
        list(disc_input.parameters()) + list(disc_output.parameters()),
        lr=Config.LEARNING_RATE, betas=(0.5, 0.999)
    )
    opt_gen = optim.Adam(
        list(gen_input.parameters()) + list(gen_output.parameters()),
        lr=Config.LEARNING_RATE, betas=(0.5, 0.999)
    )

    # Loss functions
    L1 = nn.L1Loss()
    mse = nn.MSELoss()

    dataset = FeatureDataset(features, labels)
    loader = DataLoader(dataset, batch_size=Config.BATCH_SIZE, shuffle=True)

    # Scalers for mixed precision
    g_scaler = torch.amp.GradScaler()
    d_scaler = torch.amp.GradScaler()

    # Training loop
    for epoch in range(Config.NUM_EPOCHS):
        train_fn(disc_input, disc_output, gen_input, gen_output, loader,
                 opt_disc, opt_gen, L1, mse, d_scaler, g_scaler)
        predictions_output = gen_input(torch.tensor(labels_test, dtype=torch.float32).to(Config.DEVICE))
        md = mean_deviation(torch.tensor(features_test, dtype=torch.float32).to(Config.DEVICE), predictions_output)
        print(f"Current md score: {md}")


    gen_output.eval()
    with torch.no_grad():
        predictions_output = gen_input(torch.tensor(labels_test, dtype=torch.float32).to(Config.DEVICE))
        md = mean_deviation(torch.tensor(features_test, dtype=torch.float32).to(Config.DEVICE), predictions_output)

        if md<md_score_best:
            md_score_best = md
            print(f"New best md Score: {md_score_best}")
            save_checkpoint(gen_input, opt_gen, model_save_path['gen_input'])
            save_checkpoint(gen_output, opt_gen, model_save_path['gen_output'])
            save_checkpoint(disc_input, opt_disc, model_save_path['disc_input'])
            save_checkpoint(disc_output, opt_disc, model_save_path['disc_output'])

    return md_score_best
    
# Create a study and optimize the objective function
study = optuna.create_study(direction='minimize')
study.optimize(objective, n_trials=50)

print('Best hyperparameters: ', study.best_params)

[I 2024-12-05 13:21:26,514] A new study created in memory with name: no-name-d2356788-a8cc-466f-b86e-5675ca575048
  0%|          | 0/190 [00:00<?, ?it/s]/opt/conda/lib/python3.10/site-packages/torch/nn/modules/instancenorm.py:88: UserWarning: input's size at dim=0 does not match num_features. You can silence this warning by not passing in num_features, which is not used because affine=False
  warnings.warn(f"input's size at dim={feature_dim} does not match num_features. "
100%|██████████| 190/190 [00:03<00:00, 62.49it/s]


Current md score: 0.17568817734718323


100%|██████████| 190/190 [00:03<00:00, 61.94it/s]


Current md score: 0.15824446082115173


100%|██████████| 190/190 [00:02<00:00, 63.86it/s]


...

100%|██████████| 59/59 [00:00<00:00, 61.51it/s]


Current md score: 0.2014463245868683


100%|██████████| 59/59 [00:00<00:00, 62.42it/s]


Current md score: 0.1839303970336914


 36%|███▌      | 21/59 [00:00<00:00, 63.11it/s]